In [2]:
import numpy as np
import os, sys, scipy
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
from matplotlib import rc
import scikits.bootstrap as bootstrap  
from scipy.stats.stats import spearmanr, ttest_1samp, zscore, ttest_ind, ttest_rel
from necessary_analysis_scripts import resampling_statistics,  run_stats_onetail, run_stats_twotail
from necessary_analysis_scripts import prettify_plot, calculate_aprime, load_data
from necessary_analysis_scripts import bin_classify, plot_classify
from necessary_analysis_scripts import run_nonparam_stats_onetail
import scipy.io as sio
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from datetime import date
from datetime import date
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import warnings
import mne
import pickle
warnings.simplefilter('ignore')

# Plotting 

In [3]:
#plot within jupyter notebooks
%matplotlib inline 

#tab completion for files
%config IPCompleter.greedy=True 

#supress scientific notation
np.set_printoptions(suppress=True) 

#font defaults
plt.rcParams.update({'font.size': 24})
rc('text', usetex=False)
plt.rcParams['pdf.fonttype'] = 42
if os.path.isfile("/Library/Fonts/HelveticaNeue-Light.ttf"): 
    prop = fm.FontProperties(fname="/Library/Fonts/HelveticaNeue-Light.ttf",size=24)
else:
    prop = fm.FontProperties(size=24)

#color defaults
col_cue = (60/255.,83/255.,164/255.)
col_uncue = (219/255.,11/255.,132/255.)

col_cue_acc = [6/255.,0,128/255.]
col_cue_inacc = [157/255.,153/255.,255/255.]
col_uncue_acc = [128/255.,13/255.,86/255.]
col_uncue_inacc = [255/255.,163/255.,221/255.]

b=[11/255.,0/255.,255/255.]
m=[220/255.,11/255.,144/255.]
g=[0/255.,220/255.,104/255.]
o=[245/255.,128/255.,37/155.]

# Multivariate classification information

In [4]:
nits=100
t0 = np.arange(-300,1501,10)
t = np.arange(-300,1501)
nt = np.size(t0)
tw = 100
clf = LogisticRegression(C=1,solver='liblinear',multi_class='ovr')

t0 = np.arange(-300,1501,10)
nt = np.size(t0)
iter_freqs = [
    (4,7),
    (8,12),
    (13,16),
    (16,20),
    (20,25),
    (25,30)
]

alpha = [(8,12)]

# Project details

In [ ]:
#eeg data files
project_name = 'rtPreStim02' #project directory 
#project_dir = '/Users/megan/Documents/projects/' + project_name + '/' #project directory 
project_dir = '/Volumes/Megabyte/' + project_name + '/' #project directory 
os.chdir(project_dir)

#behavior files
exp_name = 'expt2'
exp_dir = '/Users/megan/Dropbox/writing/articles/2019_rtPreStim/' + exp_name + '/'
eeg_dir = '/Users/megan/Dropbox/writing/articles/2019_rtPreStim/expt2_eeg/'

#subject files
subj_name = ['0626171_rtPreStim02','0627171_rtPreStim02','0628171_rtPreStim02',
             '0629171_rtPreStim02','0710171_rtPreStim02','0719171_rtPreStim02',
             '0720171_rtPreStim02','0725171_rtPreStim02','0803171_rtPreStim02',
             '0809171_rtPreStim02','0816171_rtPreStim02','0821171_rtPreStim02',
             '0824171_rtPreStim02','0828171_rtPreStim02','0829171_rtPreStim02',
             '0830171_rtPreStim02','0830172_rtPreStim02','0901171_rtPreStim02',
             '0904171_rtPreStim02','0906171_rtPreStim02','0907171_rtPreStim02',
             '0908171_rtPreStim02','0915171_rtPreStim02','0919171_rtPreStim02',
             '0922171_rtPreStim02','1005171_rtPreStim02','1010171_rtPreStim02',
             '1011171_rtPreStim02','1013171_rtPreStim02','1102171_rtPreStim02']

#calculate total number of subjects
nsubj = np.size(subj_name)
print ('total # subjects: %d' %(nsubj))

nb = 16              #number of blocks
nt_wm_perblock = 24  #number of wm trials per block
nt_ltm_perblock = 60 #number of ltm trials per block  
nt_wm = nb*nt_wm_perblock #total number of encoding trials         
nt_ltm = nb*nt_ltm_perblock #total number of retrieval trials

## load behavioral data

In [ ]:
dat_wm = {}
dat_ltm = {}
for isubj in range(nsubj):
    dat_wm[isubj] = pd.read_csv(exp_dir + subj_name[isubj] +'_wmlog.csv')
    dat_ltm[isubj] = pd.read_csv(exp_dir + subj_name[isubj] +'_ltmlog.csv')
    


## load artifact information

In [ ]:
cue_wmorder_noarf = np.ones((nsubj,nt_wm))
for isubj in range(nsubj):
    dir_eeg = project_dir + 'subjects/' + subj_name[isubj] + '/data/eeg/'
    
    #cue time period
    fn=dir_eeg + subj_name[isubj] + '_EEG_SEG_CLEAN.mat'
    if os.path.isfile(fn):
        mat_contents = sio.loadmat(fn,struct_as_record=False,squeeze_me=True)
        erp = mat_contents['erp']
        idx_eeg_arf = np.where(erp.arf.artifactIndCleaned==1)[0]
        cue_wmorder_noarf[isubj,idx_eeg_arf] = 0
    else:
        print('ERROR: no file exists for subject: ', subj_name[isubj])     


In [ ]:
cue_wmresperr = np.zeros((nsubj,nt_wm))
cue_ltmresperr = np.zeros((nsubj,nt_wm))
cue_ltmh = np.zeros((nsubj,nt_wm))
for isubj in range(nsubj):
    for ib in range(nb):        
        #identify ltm trials for this block (ranges from 0-60, total 60 trials)
        itrials_block = np.where(dat_ltm[isubj].block==ib)[0]
        
        #identify which of those trials were probed (ranges from 0-60, total 24 trials)
        itrials_probe = np.where(np.logical_or(dat_ltm[isubj].ltmTrialsOldValidCued[itrials_block]==1,dat_ltm[isubj].ltmTrialsOldInvalidTested[itrials_block]==1))[0]

        #save the trial number for this block (ranges from 0-384 for wm, 0-960 for ltm)
        itrials_wm = ib*nt_wm_perblock+np.arange(nt_wm_perblock)
        itrials_ltm = ib*nt_ltm_perblock+np.arange(nt_ltm_perblock)

        #select the memory performance variables for trials from this block (probed trials only)
        respdiff_wm = np.ravel(dat_wm[isubj].wmresptestimgdiff[itrials_wm])
        respdiff_ltm = np.ravel(dat_ltm[isubj].ltmresptestimgdiff[itrials_ltm])[itrials_probe]
        h_ltm = np.ravel(dat_ltm[isubj].ltmrating[itrials_ltm])[itrials_probe]==1
        
        #find the wm trial num for the tested trials
        itrials_ltm_wmtrialnum = np.ravel(dat_ltm[isubj].ltmWMtrialNum[itrials_ltm])[itrials_probe]

        #loop through all trials in this block
        for it in range(nt_wm_perblock):
            
            #find this trial order
            it_wmorder = np.where(itrials_ltm_wmtrialnum==it)[0][0]
            
            cue_wmresperr[isubj,itrials_wm[it]] = respdiff_wm[it]
            cue_ltmresperr[isubj,itrials_wm[it]] = respdiff_ltm[it_wmorder]
            cue_ltmh[isubj,itrials_wm[it]] = h_ltm[it_wmorder]

In [ ]:
def bin_classify_probcoef(X,y,clf,t=np.arange(-300,1501),t0 = np.arange(-300,1501,10),tw=100,nits=100,nbins=3,iclass=1,shuff=True,scaler=True,trainprop=.5):
    
    acc_bin = np.empty((nits,np.size(t0)))
    coef_bin = np.empty((nits,np.size(t0),np.size(np.unique(y)),np.shape(X)[1]))
    proba_bin = np.empty((nits,np.size(t0),np.size(np.unique(y)),np.size(np.unique(y))))
    acc_bin_shuff = np.empty((nits,np.size(t0)))
    
    cv = StratifiedShuffleSplit(n_splits=nits)
    scaler = StandardScaler()
    
    nlabels=np.size(np.unique(y))
    nmin = np.min([np.sum(y==i) for i in np.arange(0,nlabels)])
    nperbin = int(np.floor(nmin/nbins))
    nmin = nperbin*nbins
    for it_counter in range(nits):

        y_bin = np.sort(np.tile((np.unique(y)),nbins))
        
        #balance sizes 
        X_bin = np.zeros((np.size(y_bin),np.shape(X)[1],np.shape(X)[2]))

        counter = 0
        for i,ilabel in enumerate(np.unique(y)):
            temp=np.where(y==ilabel)[0]
            np.random.shuffle(temp)
            temp = temp[:nmin] #downsample
            if nbins != 2:
                temp_binned_labels = np.reshape(temp,(nbins,nperbin))
            else:
                if trainprop==.5:
                    temp_binned_labels = np.reshape(temp,(nbins,nperbin))
                else:
                    temp_binned_labels = {}
                    i = int(np.floor(trainprop*nmin))
                    temp_binned_labels[0] = temp[:i]
                    temp_binned_labels[1] = temp[i:]
            for ibin in range(nbins):
                X_bin[counter] = np.mean(X[temp_binned_labels[ibin]],axis=0)
                counter=counter+1
        
        if nbins==2:
            itrain = np.sort(np.arange(0,nlabels*nbins,nbins))
            itest = np.arange(1,nlabels*nbins,nbins)            
        elif nbins==3:
            itrain = np.sort(np.append(np.arange(0,nlabels*nbins,nbins),np.arange(1,nlabels*nbins,nbins)))
            itest = np.arange(2,nlabels*nbins,nbins)
        elif nbins==4:
            itrain = np.sort(np.append(np.append(np.arange(0,nlabels*nbins,nbins),np.arange(1,nlabels*nbins,nbins)),np.arange(2,nlabels*nbins,nbins)))
            itest = np.arange(3,nlabels*nbins,nbins)
        elif nbins==6: 
            itrain = np.sort(np.append(np.append(np.append(np.arange(0,nlabels*nbins,nbins),np.arange(1,nlabels*nbins,nbins)),np.arange(2,nlabels*nbins,nbins)),np.arange(3,nlabels*nbins,nbins)))
            itest = np.sort(np.append(np.arange(4,nlabels*nbins,nbins),np.arange(5,nlabels*nbins,nbins)))
        else:
            print("only coded for nbins 2, 3, 4 or 6")
        
        for n,tstart in enumerate(t0):
            #find the indices of the appropriate time window
            idx = np.where(np.logical_and(t>=tstart,t<(tstart+tw)))[0]
            X_train = np.mean(X_bin[itrain][:,:,idx],axis=2)
            X_test = np.mean(X_bin[itest][:,:,idx],axis=2)
            
            #standard scaler
            scaler.fit(X_train)
            scaler.transform(X_train)
            scaler.transform(X_test)
            
            y_train = y_bin[itrain]
            y_test = y_bin[itest]
            if shuff:
                y_train_shuff = y_bin[itrain][np.random.permutation(np.size(y_train))]
                y_test_shuff = y_bin[itest][np.random.permutation(np.size(y_test))]

            #train model
            clf.fit(X_train,y_train)
            
            #test model
            acc_bin[it_counter,n] = clf.score(X_test,y_test)
            
            #coef
            #print(clf.coef_)
            #print(np.shape(clf.coef_))
            coef_bin[it_counter,n] = clf.coef_
            
            #proba
            proba_bin[it_counter,n] = clf.predict_proba(X_test)
            if shuff:
                clf.fit(X_train,y_train_shuff)
                acc_bin_shuff[it_counter,n] = clf.score(X_test,y_test_shuff)    

    return acc_bin, coef_bin, proba_bin, acc_bin_shuff

# Can we predict trial-by-trial fluctuations of long-term memory?

In [ ]:
nits=2
acc_sustattn = np.zeros((nsubj,nits,nt))
acc_sustattn_shuff = copy.deepcopy(acc_ltmresperr_cuev)

for isubj in np.arange(nsubj):
    print(isubj)
    
    #load erp
    dir_eeg = project_dir + 'subjects/' + subj_name[isubj] + '/data/eeg/'
    fn=dir_eeg + subj_name[isubj] + '_EEG_SEG_CLEAN.mat'
    mat_contents = sio.loadmat(fn,struct_as_record=False,squeeze_me=True)
    erp = mat_contents['erp']

    #calculate alpha power
    info = mne.create_info(erp.chanLabels[:30].tolist(),1000,'eeg')
    epochs = mne.EpochsArray(erp.trial.baselined[erp.arf.artifactIndCleaned==0,:30],info=info,tmin=-1.1,baseline=None)
    
    counter=0
    for fmin, fmax in iter_freqs:
        raw_bp = epochs.copy()
        raw_bp.filter(fmin,fmax,n_jobs=1,l_trans_bandwidth=1,h_trans_bandwidth=1)
        raw_bp.apply_hilbert(envelope=True)
        raw_bp.crop(-.3,1.5)

        if counter == 0:
            eegs_allfreq = raw_bp._data/np.mean(raw_bp._data[:,:,:300])
        else:
            eegs_allfreq = np.append(eegs_allfreq,raw_bp._data/np.mean(raw_bp._data[:,:,:300]),axis=1)

        counter = counter+1
    
    #classify mne data 
    w=np.floor(np.ravel(dat_wm[isubj].cueposbin[cue_wmorder_noarf[isubj]==1]/2))
    x=np.abs(cue_ltmresperr[isubj][cue_wmorder_noarf[isubj]==1])
    cuev = (dat_wm[isubj].cuevalid==1)[cue_wmorder_noarf[isubj]==1]
    y = [np.median(x[w==i]) for i in range(4)] 
    y_cuev = [np.median(x[np.logical_and(w==i,cuev)]) for i in range(4)] 
    y_cuei = [np.median(x[np.logical_and(w==i,cuev==0)]) for i in range(4)] 
    z = np.zeros(np.shape(w))
    for i in range(4):
        z[np.logical_and(w==i,x<y[i])] = 0
    for i in range(4):
        z[np.logical_and(w==i,x>=y[i])] = 1
    print(z)
    labels = z
    
    acc_sustattn[isubj],_,_,acc_sustattn_shuff[isubj] = bin_classify_probcoef(eegs_allfreq,labels,clf,nits=nits,tw=tw,t0=t0,nbins=2)
    
    df = pd.DataFrame(acc_sustattn[isubj])
    df.columns = t0
    df.to_csv(eeg_dir + subj_name[isubj] + "_sustattn_" + date.today().strftime('%Y%m%d') + ".csv",index=True,header=True)

    df = pd.DataFrame(acc_sustattn_shuff[isubj])
    df.columns = t0
    df.to_csv(eeg_dir + subj_name[isubj] + "_sustattn_" + date.today().strftime('%Y%m%d') + ".csv",index=True,header=True)
    

In [ ]:
#statistics
temp_p = np.empty(nt)
for iit in range(nt):
    temp_p[iit],_ = run_stats_onetail(np.mean(acc_ltmresperr_cuev*100,axis=1)[:,iit],50)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6))
ax.fill_between(t0,np.mean(np.mean(acc_sustattn*100,axis=1),axis=0)-np.std(np.mean(acc_sustattn*100,axis=1),axis=0)/np.sqrt(nsubj),
                 np.mean(np.mean(acc_sustattn*100,axis=1),axis=0)+np.std(np.mean(acc_sustattn*100,axis=1),axis=0)/np.sqrt(nsubj),
                color='blue',alpha=.25)
ax.plot(t0,np.mean(np.mean(acc_sustattn*100,axis=0),axis=0),color='blue')
ax.fill_between(t0,np.mean(np.mean(acc_sustattn_shuff*100,axis=1),axis=0)-np.std(np.mean(acc_sustattn_shuff*100,axis=1),axis=0)/np.sqrt(nsubj),
                np.mean(np.mean(acc_sustattn_shuff*100,axis=1),axis=0)+np.std(np.mean(acc_sustattn_shuff*100,axis=1),axis=0)/np.sqrt(nsubj),
               color='k',alpha=.25)
ax.plot(t0,np.mean(np.mean(acc_sustattn_shuff*100,axis=0),axis=0),color='k',alpha=.5)
thr=.05
#plt.scatter(t0[temp_p<thr],np.ones(nt)[temp_p<thr]*46,marker='s',color='k')
#plt.scatter(t0[temp_p>(1-thr)],np.ones(nt)[temp_p>(1-thr)]*46,marker='s',color='k')
prettify_plot(ax,xlim=[-300,1500],ylim=[30,70],
             xt=[-300,0,300,600,900,1200,1500],
             xtl=[-300,0,300,600,900,1200,1500],
             yt=[30,40,50,60,70],
             ytl=[30,40,50,60,70],
             xl='Time relative to cue (ms)',
             yl='Classifier accuracy (%)')

# Do long-term memory fluctuations reflect differences in spatial attention? 

In [ ]:
nits=1000
t0 = np.arange(-300,1501,10)
nt = np.size(t0)
n = copy.deepcopy(nsubj)
s = copy.deepcopy(subj_name)
iter_freqs = [
    (8,12),
]
acc_cuepos_traccinacc_testinacc = np.zeros((n,nits,nt))
acc_cuepos_traccinacc_testinacc_shuff = copy.deepcopy(acc_cuepos_inacctrials)
acc_cuepos_traccinacc_testacc = np.zeros((n,nits,nt))
acc_cuepos_traccinacc_testacc_shuff = copy.deepcopy(acc_cuepos_inacctrials)
scaler = StandardScaler()

for isubj in np.arange(n):
    print(isubj)
    
    #load erp
    dir_eeg = project_dir + 'subjects/' + subj_name[isubj] + '/data/eeg/'
    fn=dir_eeg + subj_name[isubj] + '_EEG_SEG_CLEAN.mat'
    mat_contents = sio.loadmat(fn,struct_as_record=False,squeeze_me=True)
    erp = mat_contents['erp']

    #calculate alpha power
    info = mne.create_info(erp.chanLabels[:30].tolist(),1000,'eeg')
    epochs = mne.EpochsArray(erp.trial.baselined[erp.arf.artifactIndCleaned==0,:30],info=info,tmin=-1.1,baseline=None)
    
    counter=0
    for fmin, fmax in iter_freqs:
        raw_bp = epochs.copy()
        raw_bp.filter(fmin,fmax,n_jobs=1,l_trans_bandwidth=1,h_trans_bandwidth=1)
        #raw_bp.subtract_evoked()
        raw_bp.apply_hilbert(envelope=True)
        #mne.baseline.rescale(raw_bp._data,raw_bp.times, (None,0), mode='zscore',copy=False)
        raw_bp.crop(-.3,1.5)

        if counter == 0:
            eegs_allfreq = raw_bp._data
        else:
            eegs_allfreq = np.append(eegs_allfreq,raw_bp._data,axis=1)

        counter = counter+1
    
    #classify mne data 
    w=np.floor(np.ravel(dat_wm[isubj].cueposbin[cue_wmorder_noarf[isubj]==1]/2))
    x=np.abs(cue_ltmresperr[isubj][cue_wmorder_noarf[isubj]==1])
    cuev = (dat_wm[isubj].cuevalid==1)[cue_wmorder_noarf[isubj]==1]
    y_cuev = [np.median(x[np.logical_and(w==i,cuev)]) for i in range(4)] 
    z = np.zeros(np.sum(cuev))
    for i in range(4):
        z[np.logical_and(w[cuev==1]==i,x[cuev==1]<y_cuev[i])] = 0
    for i in range(4):
        z[np.logical_and(w[cuev==1]==i,x[cuev==1]>=y_cuev[i])] = 1
    print(z)
    
    #
    labels_acc = w[cuev==1][z==0]
    eegs_acc = eegs_allfreq[cuev==1][z==0]
    labels_inacc = w[cuev==1][z==1]
    eegs_inacc = eegs_allfreq[cuev==1][z==1]
    
    nbins=2
    nlabels_acc=np.size(np.unique(labels_acc))
    nmin_acc = np.min([np.sum(labels_acc==i) for i in np.arange(0,nlabels_acc)])
    nperbin_acc = int(np.floor(nmin_acc/nbins))
    nlabels_inacc=np.size(np.unique(labels_inacc))
    nmin_inacc = np.min([np.sum(labels_inacc==i) for i in np.arange(0,nlabels_inacc)])
    nperbin_inacc = int(np.floor(nmin_inacc/nbins))
    nperbin = np.min([nperbin_acc,nperbin_inacc])
    nmin = nperbin*nbins
    
    labels_acc_bin = np.sort(np.tile((np.unique(labels_acc)),nbins))
    labels_inacc_bin = np.sort(np.tile((np.unique(labels_inacc)),nbins))
        
    for it_counter in range(nits):

        #balance sizes 
        eegs_acc_bin = np.zeros((np.size(labels_acc_bin),np.shape(eegs_acc)[1],np.shape(eegs_acc)[2]))
        eegs_inacc_bin = np.zeros((np.size(labels_inacc_bin),np.shape(eegs_inacc)[1],np.shape(eegs_inacc)[2]))

        #bin trials
        counter = 0
        for i,ilabel in enumerate(np.unique(labels_acc)):
            temp=np.where(labels_acc==ilabel)[0]
            np.random.shuffle(temp)
            temp = temp[:nmin] #downsample
            temp_binned_labels = np.reshape(temp,(nbins,nperbin))
            
            for ibin in range(nbins):
                eegs_acc_bin[counter] = np.mean(eegs_acc[temp_binned_labels[ibin]],axis=0)
                counter=counter+1
        
        counter = 0
        for i,ilabel in enumerate(np.unique(labels_inacc)):
            temp=np.where(labels_inacc==ilabel)[0]
            np.random.shuffle(temp)
            temp = temp[:nmin] #downsample
            temp_binned_labels = np.reshape(temp,(nbins,nperbin))
            
            for ibin in range(nbins):
                eegs_inacc_bin[counter] = np.mean(eegs_inacc[temp_binned_labels[ibin]],axis=0)
                counter=counter+1
        
        eegs_train = np.mean(np.append(eegs_acc_bin[np.arange(0,counter,nbins)][np.newaxis],eegs_inacc_bin[np.arange(0,counter,nbins)][np.newaxis],axis=0),axis=0)
        labels_train = labels_acc_bin[np.arange(0,counter,nbins)]
        
        eegs_test_acc = eegs_acc_bin[np.arange(1,counter,nbins)]
        eegs_test_inacc = eegs_inacc_bin[np.arange(1,counter,nbins)]
        labels_test_acc = labels_acc_bin[np.arange(1,counter,nbins)]
        labels_test_inacc = labels_inacc_bin[np.arange(1,counter,nbins)]
    
        for n,tstart in enumerate(t0):
            #find the indices of the appropriate time window
            idx = np.where(np.logical_and(t>=tstart,t<(tstart+tw)))[0]
            X_train = np.mean(eegs_train[:,:,idx],axis=2)
            X_test_acc = np.mean(eegs_test_acc[:,:,idx],axis=2)
            X_test_inacc = np.mean(eegs_test_inacc[:,:,idx],axis=2)
            
            #standard scaler
            scaler.fit(X_train)
            scaler.transform(X_train)
            scaler.transform(X_test_acc)
            scaler.transform(X_test_inacc)

            #train model
            clf.fit(X_train,labels_train)
            
            #test model
            acc_cuepos_traccinacc_testacc[isubj,it_counter,n] = clf.score(X_test_acc,labels_test_acc)
            acc_cuepos_traccinacc_testinacc[isubj,it_counter,n] = clf.score(X_test_inacc,labels_test_inacc)
    
    df = pd.DataFrame(acc_cuepos_traccinacc_testinacc[isubj])
    df.columns = t0
    df.to_csv(eeg_dir + subj_name[isubj] + "_acc_cuepos_traccinacc_testinacc_" + date.today().strftime('%Y%m%d') + ".csv",index=True,header=True)

    df = pd.DataFrame(acc_cuepos_traccinacc_testacc[isubj])
    df.columns = t0
    df.to_csv(eeg_dir + subj_name[isubj] + "_acc_cuepos_traccinacc_testacc_" + date.today().strftime('%Y%m%d') + ".csv",index=True,header=True)
    

In [ ]:
temp_p_acc = np.empty(nt)
for iit in range(nt):
    temp_p_acc[iit],_ = run_stats_onetail(np.mean(acc_cuepos_traccinacc_testacc,axis=1)[:,iit],.25)
    
temp_p_inacc = np.empty(nt)
for iit in range(nt):
    temp_p_inacc[iit],_ = run_stats_onetail(np.mean(acc_cuepos_traccinacc_testinacc,axis=1)[:,iit],.25)

In [ ]:
temp_p = np.empty(nt)
for iit in range(nt):
    temp_p[iit],_ = run_stats_onetail(np.mean(acc_cuepos_traccinacc_testacc,axis=1)[:,iit],np.mean(acc_cuepos_traccinacc_testinacc,axis=1)[:,iit])
    

In [ ]:
y=np.mean(np.mean(acc_cuepos_traccinacc_testacc,axis=1),axis=0)
ye=np.std(np.mean(acc_cuepos_traccinacc_testacc,axis=1),axis=0)/np.sqrt(nsubj_cue)
plt.fill_between(t0,y-ye,y+ye,alpha=.5)
plt.plot(t0,y)
y=np.mean(np.mean(acc_cuepos_traccinacc_testinacc,axis=1),axis=0)
ye=np.std(np.mean(acc_cuepos_traccinacc_testinacc,axis=1),axis=0)/np.sqrt(nsubj_cue)
plt.fill_between(t0,y-ye,y+ye,alpha=.5)
plt.plot(t0,y)
plt.scatter(t0[temp_p<.025],.5*np.ones(np.sum(temp_p<.025)),color='k')
plt.scatter(t0[temp_p_acc<.05],.49*np.ones(np.sum(temp_p_acc<.05)),color='b')
plt.scatter(t0[temp_p_inacc<.05],.48*np.ones(np.sum(temp_p_inacc<.05)),color='orange')

# WM vs LTM labels

In [ ]:
nits=1000
t0 = np.arange(-300,1501,10)
nt = np.size(t0)
n = copy.deepcopy(nsubj)
s = copy.deepcopy(subj_name)
nlabels = np.zeros((nsubj),dtype=int)
nconsistent = np.zeros((nsubj))
nconsistent_shuffle = np.zeros((nsubj,nits))
n_within2deg = np.zeros((nsubj))
thr_subj = np.zeros((nsubj))
for isubj in np.arange(nsubj):
    print(isubj)
    
    #classify mne data 
    #labels_mne = np.floor(np.ravel(dat_wm_cue[isubj].cueposbin[cue_wmorder_noarf[isbj]==1])/2)
    w=np.floor(np.ravel(dat_wm[isubj].cueposbin[cue_wmorder_noarf[isubj]==1]/2))
    x=np.abs(cue_ltmresperr[isubj][cue_wmorder_noarf[isubj]==1])
    cuev = (dat_wm[isubj].cuevalid==1)[cue_wmorder_noarf[isubj]==1]
    y = [np.median(x[w==i]) for i in range(4)] 
    y_cuev = [np.median(x[np.logical_and(w==i,cuev)]) for i in range(4)] 
    y_cuei = [np.median(x[np.logical_and(w==i,cuev==0)]) for i in range(4)] 
    z = np.zeros(np.shape(w))
    for i in range(4):
        z[np.logical_and(w==i,x<y[i])] = 0
        #z[np.logical_and(np.logical_and(w==i,cuev==1),x<y_cuev[i])] = 0
        #z[np.logical_and(np.logical_and(w==i,cuev==0),x<y_cuei[i])] = 0
    for i in range(4):
        z[np.logical_and(w==i,x>=y[i])] = 1
        #z[np.logical_and(np.logical_and(w==i,cuev==1),x>=y_cuev[i])] = 1
        #z[np.logical_and(np.logical_and(w==i,cuev==0),x>=y_cuei[i])] = 1
    #print(z)
    labels = z
    x_ltm = copy.deepcopy(x)
    z_ltm = copy.deepcopy(z)
    
    #classify mne data 
    #labels_mne = np.floor(np.ravel(dat_wm_cue[isubj].cueposbin[cue_wmorder_noarf[isbj]==1])/2)
    w=np.floor(np.ravel(dat_wm[isubj].cueposbin[cue_wmorder_noarf[isubj]==1]/2))
    x=np.abs(cue_wmresperr[isubj][cue_wmorder_noarf[isubj]==1])
    cuev = (dat_wm[isubj].cuevalid==1)[cue_wmorder_noarf[isubj]==1]
    y = [np.median(x[w==i]) for i in range(4)] 
    y_cuev = [np.median(x[np.logical_and(w==i,cuev)]) for i in range(4)] 
    y_cuei = [np.median(x[np.logical_and(w==i,cuev==0)]) for i in range(4)] 
    z = np.zeros(np.shape(w))
    
    for i in range(4):
        z[np.logical_and(w==i,x<y[i])] = 0
        #z[np.logical_and(np.logical_and(w==i,cuev==1),x<y_cuev[i])] = 0
        #z[np.logical_and(np.logical_and(w==i,cuev==0),x<y_cuei[i])] = 0
    for i in range(4):
        z[np.logical_and(w==i,x>=y[i])] = 1
        #z[np.logical_and(np.logical_and(w==i,cuev==1),x>=y_cuev[i])] = 1
        #z[np.logical_and(np.logical_and(w==i,cuev==0),x>=y_cuei[i])] = 1
    #print(z)
    thr_wm = np.zeros(np.shape(w))
    for ii in range(np.size(w)):
        thr_wm[ii] = y[int(w[ii])]
    n_within2deg[isubj] = np.sum(np.abs(x-thr_wm)<2)
    labels_wm = z
    thr_subj[isubj] = np.mean(y)
    
    nlabels[isubj] = np.size((labels_wm-labels))
    nconsistent[isubj] = np.sum((labels_wm==labels))
    
    for iit in range(nits):
        nconsistent_shuffle[isubj,iit] = np.sum((labels_wm[np.random.permutation(nlabels[isubj])]==labels))
    

In [ ]:
thr_subj[isubj]

In [ ]:
print(np.mean(nconsistent/nlabels))
print(np.std(nconsistent/nlabels))